In [1]:
import pandas as pd
import numpy as np

# World Bank Worldwide Governance Indicators
- <b>Goal</b>: Compile the relevant World Bank Governance indicators for further regression analysis
- The dataset covers the dataset from 215 countries bwtween 1996-2021.
- <b>Assumptions</b>:
    - Missing data in 1997, 1999, 2001 are filled as average of years prior and after

In [ ]:
WGI_dir = "../../data/global_WB_WorldwideGovernance/"

In [2]:
all_data = pd.read_csv(WGI_dir+"P_Data_Extract_From_Worldwide_Governance_Indicators/bade4c16-2b40-4148-a71d-bfc5b99df2d5_Data.csv")
all_data

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.29170477390289,-1.17601227760315,-1.27172422409058,-1.25113701820374,-1.34417974948883,-1.35064733028412,...,-1.43037283420563,-1.44590842723846,-1.36474287509918,-1.3485871553421,-1.53404927253723,-1.52172565460205,-1.48977172374725,-1.40771043300629,-1.48053777217865,-1.14237689971924
1,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0,0,0,0,0,0,...,0,0,0,0.480769217014313,0,0,0,1.44230771064758,0,4.807692527771
2,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.4193553924561,33.6898384094238,30.8510646820068,32.8042335510254,19.0476188659668,15.2709360122681,...,11.8483409881592,9.95260620117188,12.5,12.0192308425903,9.61538505554199,9.61538505554199,9.61538505554199,11.057692527771,10.5769233703613,21.1538467407227
3,Afghanistan,AFG,Control of Corruption: Standard Error,CC.STD.ERR,0.340506970882416,0.324012994766235,0.346905559301376,0.352838128805161,0.270214706659317,0.213201358914375,...,0.186088413000107,0.159914895892143,0.161357954144478,0.160177767276764,0.164447620511055,0.161955490708351,0.155046001076698,0.15508745610714,0.157702177762985,0.183094143867493
4,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,...,10,11,11,11,10,10,10,10,9,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7707,Data from database: Worldwide Governance Indic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
series = ["Government Effectiveness: Estimate","Regulatory Quality: Estimate", "Voice and Accountability: Estimate"]
name_list = ["WG_GovEff","WG_Reg","WG_Voice"]
for series_select in series:
    name_select = name_list[series.index(series_select)]
    data_select = all_data[all_data["Series Name"]==series_select]
    data_select_melt = pd.melt(data_select, id_vars=['Country Code'], value_vars=all_data.columns[4:],var_name='year_raw', value_name=name_select)
    data_select_melt[["year","bin"]] = data_select_melt["year_raw"].str.split(" ", expand = True)
    data_select_melt["year"] = data_select_melt["year"].astype(int)
    data_select_melt = data_select_melt.drop(columns=["bin","year_raw"])
    data_select_melt = data_select_melt.sort_values(by="year")
    data_select_melt = data_select_melt.set_index(["Country Code","year"])
    if series.index(series_select) == 0:
        data_compile = data_select_melt
    else:
        data_compile[name_select] = data_select_melt[name_select]
data_compile = data_compile.replace("..",np.nan)
data_compile = data_compile.astype(float)
data_compile = data_compile.reset_index()
data_compile

,Country Code,year,WG_GovEff,WG_Reg,WG_Voice
0,AFG,1996,-2.175167,-2.090330,-1.908540
1,NPL,1996,-0.380564,-0.401736,-0.064157
2,NLD,1996,1.942935,1.841060,1.557145
3,ANT,1996,NaN,NaN,NaN
4,NZL,1996,1.771767,1.891940,1.629558
...,...,...,...,...,...
4917,GRL,2021,0.775258,1.371155,1.288688
4918,GRD,2021,0.026532,0.291878,0.734088
4919,GUM,2021,0.697110,0.548629,0.780045
4920,GIN,2021,-0.915673,-0.964261,-0.987571


In [31]:
data_compile_assump = data_compile.copy()
countries_unique = data_compile["Country Code"].unique()
for country_select in countries_unique:
    data_compile_country = data_compile[data_compile["Country Code"]==country_select]
    years_missing = [1997,1999,2001]
    for ym in years_missing:
        ym_data = data_compile_country[data_compile_country["year"].isin([ym+1,ym-1])]
        ym_data_mean = ym_data.groupby("Country Code").mean()
        ym_data_mean = ym_data_mean.reset_index()
        data_compile_assump = pd.concat([data_compile_assump,ym_data_mean])
data_compile_assump = data_compile_assump.sort_values(by=["Country Code","year"])
data_compile_assump = data_compile_assump.set_index(["Country Code","year"])
data_compile_assump.to_csv("_all_temporal_worldwide_governance.csv")
data_compile_assump

WG_GovEff    WG_Reg  WG_Voice
Country Code year                                 
ABW          1996.0        NaN       NaN       NaN
             1997.0        NaN       NaN       NaN
             1998.0        NaN       NaN       NaN
             1999.0        NaN       NaN       NaN
             2000.0        NaN       NaN       NaN
...                        ...       ...       ...
ZWE          2017.0  -1.238663 -1.564137 -1.195904
             2018.0  -1.261774 -1.509489 -1.140975
             2019.0  -1.267288 -1.468613 -1.164705
             2020.0  -1.299779 -1.416232 -1.113716
             2021.0  -1.242930 -1.372497 -1.136934

[5564 rows x 3 columns]